# ダッシュボード作成

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.express as px

from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table

pd.set_option("display.max_rows",99999)
pd.set_option("display.max_columns",99999)

In [2]:
# データの読み込み
df = pd.read_csv("./data.csv", encoding="shift-jis")
df.head()

,表章項目コード,表章項目,品目分類コード,品目分類,世帯区分(二人以上の世帯)コード,世帯区分(二人以上の世帯),地域区分コード,地域区分,時間軸(年次)コード,時間軸(年次),/世帯主の年齢階級,平均,29歳以下,30〜39歳,40〜49歳,50〜59歳,60〜69歳,70歳以上,25〜34歳,35〜44歳,45〜54歳,55〜64歳,65歳以上
0,1,金額,100000,世帯数分布(抽出率調整)【一万分比】,3,二人以上の世帯(2000年〜),0,全国,2000000000,2000年,NaN,"10,000",385,"1,637","2,138","2,427","2,131","1,282",NaN,NaN,NaN,NaN,NaN
1,1,金額,100000,世帯数分布(抽出率調整)【一万分比】,3,二人以上の世帯(2000年〜),0,全国,2001000000,2001年,NaN,"10,000",366,"1,573","1,961","2,410","2,310","1,379",NaN,NaN,NaN,NaN,NaN
2,1,金額,100000,世帯数分布(抽出率調整)【一万分比】,3,二人以上の世帯(2000年〜),0,全国,2002000000,2002年,NaN,"10,000",354,"1,531","1,915","2,489","2,237","1,475",NaN,NaN,NaN,NaN,NaN
3,1,金額,100000,世帯数分布(抽出率調整)【一万分比】,3,二人以上の世帯(2000年〜),0,全国,2003000000,2003年,NaN,"10,000",324,"1,562","1,887","2,434","2,286","1,507",NaN,NaN,NaN,NaN,NaN
4,1,金額,100000,世帯数分布(抽出率調整)【一万分比】,3,二人以上の世帯(2000年〜),0,全国,2004000000,2004年,NaN,"10,000",294,"1,497","1,982","2,325","2,275","1,626",NaN,NaN,NaN,NaN,NaN


In [3]:
df2 = pd.read_csv('./data2.csv')
df2.head()

,時間軸(年次),年代,消費支出【円】
0,2000年,29歳以下,"3,064,513"
1,2000年,30〜39歳,"3,457,389"
2,2000年,40〜49歳,"4,273,926"
3,2000年,50〜59歳,"4,419,172"
4,2000年,60〜69歳,"3,605,165"


In [4]:
df.shape

(31210, 23)

In [5]:
#欠損値の確認
df.isna().sum()

表章項目コード                 0
表章項目                    0
品目分類コード                 0
品目分類                    0
世帯区分(二人以上の世帯)コード        0
世帯区分(二人以上の世帯)           0
地域区分コード                 0
地域区分                    0
時間軸(年次)コード              0
時間軸(年次)                 0
/世帯主の年齢階級           31210
平均                      0
29歳以下                   0
30〜39歳                  0
40〜49歳                  0
50〜59歳                  0
60〜69歳                  0
70歳以上                   0
25〜34歳              20170
35〜44歳              20170
45〜54歳              20170
55〜64歳              20170
65歳以上               20170
dtype: int64

In [6]:
#必要データの抽出
df = df.iloc[:,:-5]

In [7]:
# df1から必要データ（品目区分（消費支出）と世帯区分(二人以上の世帯)）の抽出
df1 = df[df["品目分類"]=='消費支出']
df1 = df1[df1['世帯区分(二人以上の世帯)コード'] == 3]
df1 = df1.iloc[:,9:]
df1.head()

,時間軸(年次),/世帯主の年齢階級,平均,29歳以下,30〜39歳,40〜49歳,50〜59歳,60〜69歳,70歳以上
460,2000年,NaN,"3,807,937","3,064,513","3,457,389","4,273,926","4,419,172","3,605,165","2,878,277"
461,2001年,NaN,"3,708,649","3,013,398","3,415,162","4,224,260","4,317,727","3,389,972","2,970,594"
462,2002年,NaN,"3,671,438","2,834,743","3,364,871","4,153,660","4,234,576","3,387,233","3,047,981"
463,2003年,NaN,"3,622,095","2,792,339","3,345,468","4,151,392","4,134,545","3,415,481","2,909,979"
464,2004年,NaN,"3,635,703","3,046,971","3,378,362","4,126,574","4,215,410","3,377,241","2,917,288"


In [8]:
# 2人以上世帯の消費品目の抽出(大分類である10個を抽出)
df3 = df.query('品目分類==["1食料","2住居","3光熱・水道","4家具・家事用品","5被服","6保健医療","7交通・通信","8教育","9教養娯楽","10その他"]')
df3 = df3[df3['世帯区分(二人以上の世帯)コード'] == 3]
df3.head()

,表章項目コード,表章項目,品目分類コード,品目分類,世帯区分(二人以上の世帯)コード,世帯区分(二人以上の世帯),地域区分コード,地域区分,時間軸(年次)コード,時間軸(年次),/世帯主の年齢階級,平均,29歳以下,30〜39歳,40〜49歳,50〜59歳,60〜69歳,70歳以上
506,1,金額,10000000,1食料,3,二人以上の世帯(2000年〜),0,全国,2000000000,2000年,NaN,"973,680","625,283","822,885","1,072,559","1,080,639","1,001,918","855,127"
507,1,金額,10000000,1食料,3,二人以上の世帯(2000年〜),0,全国,2001000000,2001年,NaN,"945,571","610,632","811,939","1,046,013","1,054,108","942,210","858,196"
508,1,金額,10000000,1食料,3,二人以上の世帯(2000年〜),0,全国,2002000000,2002年,NaN,"939,218","585,445","799,423","1,031,023","1,036,852","951,732","865,191"
509,1,金額,10000000,1食料,3,二人以上の世帯(2000年〜),0,全国,2003000000,2003年,NaN,"919,666","592,002","769,559","1,004,286","1,002,084","952,096","857,262"
510,1,金額,10000000,1食料,3,二人以上の世帯(2000年〜),0,全国,2004000000,2004年,NaN,"914,712","590,663","768,853","977,150","1,021,838","938,837","844,118"


In [9]:
# # 単身世帯のデータ読み込み、消費品目の抽出
# df_alone = pd.read_csv("./data1.csv", encoding="shift-jis")
# df_alone = df_alone[["時間軸(年次)","1食料【円】","2住居【円】","3光熱・水道【円】","4家具・家事用品【円】","5被服及び履物【円】","6保健医療【円】","7交通・通信【円】","8教育【円】","9教養娯楽【円】","10その他の消費支出【円】"]]
# df_alone = df_alone.set_index('時間軸(年次)')
# df_alone

In [10]:
# dashアプリケーションの初期化
app = JupyterDash(__name__)

# dashアプリケーションの定義
app.layout = html.Div([
        html.H1("2人以上世帯の年間消費支出額",
        style={"background-color":"rgb(0, 0, 128)",
               "color":"white"}),
        html.Div([
                html.H2("年間消費支出額の推移"),
                dcc.Graph(
                    id="折れ線グラフ1",
                    figure={
                        "data": [
                            go.Scatter(
                                x=df1["時間軸(年次)"],
                                y=df1["平均"]
                            )],
                        "layout": go.Layout(
                            title="年次毎の年間消費支出額",
                            xaxis={"title": "年"},
                            yaxis={"title": "消費支出額(円)", "tickformat": ",.0f"},
                            ),},
                ),
            ],
        ),
        html.Div([
                html.H2("年代別の年間消費支出額"),
                dcc.Dropdown(
                    id="dropdown1",
                    options=[{"label": x, "value": x} for x in df2["時間軸(年次)"].unique()],
                    style={"width": "320px"},
                    multi=False,
                    placeholder="年次",),
                dcc.Graph(id="折れ線グラフ2"),],
            style={"width": "50%", "display": "inline-block"},
        ),
        html.Div([
                html.H2("年間消費支出の内訳"),
                dcc.Dropdown(
                    id="dropdown2",
                    options=[{"label": x, "value": x} for x in df3["時間軸(年次)"].unique()],
                    style={"width": "320px"},
                    multi=False,
                    placeholder="年次",),
                dcc.Graph(id="棒グラフ"),],
            style={"width": "50%", "display": "inline-block"},
        ),
      
    ])


In [11]:
from dash.dependencies import Input, Output
@app.callback(Output('折れ線グラフ2', 'figure'),
              [Input('dropdown1', 'value')])
def update_graph(selected_year):
    traces = []  
    if selected_year:
        filtered_df = df2[df2['時間軸(年次)'] == selected_year]
        if not filtered_df.empty:
            traces.append(go.Bar(x=filtered_df['年代'],
                                 y=filtered_df['消費支出【円】']
                                     ))
            return {
                'data': traces,
                'layout': go.Layout(title="年代毎の年間消費支出額",
                                    xaxis={"title": "年代"},
                                    yaxis={"title": "消費支出額(円)", "tickformat": ",.0f"})
            }
        else:
            return {
                'data': traces, 
                'layout': go.Layout(title="選択された年次のデータが存在しません")
            }
    else:
        return {
            'data': traces, 
            'layout': go.Layout(title="年次を選択してください")
        }
@app.callback(Output('棒グラフ', 'figure'),
              [Input('dropdown2', 'value')])
def update_graph(selected_year):
    traces = []
    if selected_year:
        filtered_dff = df3[df3['時間軸(年次)'] == selected_year]
        filtered_dff = filtered_dff.sort_values('平均')

        traces.append(go.Bar(x=filtered_dff['平均'],
                             y=filtered_dff['品目分類'],
                             orientation='h'
        ))

    return {
        'data': traces,
        'layout': go.Layout(title="年間消費支出の内訳",
                            xaxis={"title": "消費支出額(円)", "tickformat": ",.0f"},
                            yaxis={"title": "品目"},)
    }


### サーバーの実行

In [12]:
app.run_server(mode = "inline", debug=True, port='1112')

Dash is running on http://127.0.0.1:1112/

